In [1]:
#!pip install selenium

In [1]:
# 스레드 + 크롤링
from flask import Flask, url_for, request, jsonify, redirect, Response
from flask_cors import CORS, cross_origin

import json
import requests

import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

import threading
import time
import logging

import pickle

In [2]:
CRAWLED_RES_SAVED_FILE = 'crawl_data/result'

In [4]:
# 기록용 스레드
def record_thread(cond):
    print("기록용 스레드 구동 시작!")
    cnt = 0
    
    while True:
        if crawl_fin:
            f = open(CRAWLED_RES_SAVED_FILE, 'wb')
            pickle.dump(hrefURL, f)
            f.close()
            
        time.sleep(1)

In [5]:
# 관리 스레드
def thread_manager(cond):
    print("스레드 매니저 구동 시작")
    
    while True:
        with cond:
            if crawl_fin:
                cond.notifyAll()
                
            time.sleep(1)

In [6]:
# 크롤링 스레드
def crawl_thread(cond, text):
    global crawl_fin
    crawl_fin = False
    
    print("크롤링 스레드 구동 시작!")
    
    url = "https://www.youtube.com/results?search_query="
    
    driver = webdriver.Chrome(r"C:\chromedriver.exe")
    
    search_url = url + text
    
    driver.get(url = search_url)
    
    while True:
        with cond:
            body = driver.find_element_by_tag_name('body')
            
            global hrefURL
            hrefURL = set()
            
            shelfContents = driver.find_elements_by_css_selector(
                "#contents > ytd-shelf-renderer > div > div > ytd-vertical-list-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
            )
            
            for item in shelfContents:
                if (item != ''):
                    hrefURL.add(item.get_attribute('href'))
                    
            backup_height = driver.execute_script("return document.body.scrollHeight")
                    
            while True:
                body.send_keys(Keys.END)
                time.sleep(2)
                
                current_height = driver.execute_script("return document.body.scrollHeight")
                
                if backup_height == current_height:
                    break
                    
                backup_height = current_height
                    
            allContents = driver.find_elements_by_css_selector(
                "#contents > ytd-item-section-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
            )
            
            for item in allContents:
                if (item != ''):
                    hrefURL.add(item.get_attribute('href'))
            
            print(hrefURL)
            print(len(hrefURL))
            
            crawl_fin = True
            cond.wait()
            crawl_fin = False

In [ ]:
if __name__ == '__main__':
    cond = threading.Condition()
    crawler = threading.Thread(name = 'Crawler', target = crawl_thread, args = (cond, '시고르자브종'))
    recorder = threading.Thread(name = 'Recorder', target = record_thread, args = (cond, ))
    manager = threading.Thread(name = 'Manager', target = thread_manager, args = (cond, ))
    
    crawler.start()
    time.sleep(2)
    recorder.start()
    manager.start()
    
    crawler.join()
    recorder.join()
    manager.join()

크롤링 스레드 구동 시작!
기록용 스레드 구동 시작!
스레드 매니저 구동 시작
{'https://www.youtube.com/watch?v=FR1uIeQcj1Q', 'https://www.youtube.com/watch?v=P4Lv_pq0NNY', 'https://www.youtube.com/watch?v=OMBjyeSJLqY', 'https://www.youtube.com/watch?v=fCcDvEPGtes', 'https://www.youtube.com/shorts/CsaVgPJgKtg', 'https://www.youtube.com/watch?v=c9cYIGqdcvA', 'https://www.youtube.com/watch?v=WOKChftHOUA', 'https://www.youtube.com/watch?v=wd1GTiq2K1g', 'https://www.youtube.com/watch?v=TYko_VKWSJ0', 'https://www.youtube.com/watch?v=gmd4OnrxR0Q', 'https://www.youtube.com/watch?v=FG4E9lHxEiQ', 'https://www.youtube.com/watch?v=zMkq_UpuRzo', 'https://www.youtube.com/shorts/22_1CRL35Qk', 'https://www.youtube.com/watch?v=UUtnh3J8xfQ', 'https://www.youtube.com/watch?v=5aNHqVXQ4W8', 'https://www.youtube.com/watch?v=i71J5vHvjzI', 'https://www.youtube.com/watch?v=wgPowY0G4Sw', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/watch?v=mA1sz4nbIV8', 'https://www.youtube.com/watch?v=CPjlGkw9iVE', 'https://www.yout

{'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/watch?v=q0I4f8771bc', 'https://www.youtube.com/shorts/CsaVgPJgKtg', 'https://www.youtube.com/watch?v=PRikXG8Mcrg', 'https://www.youtube.com/watch?v=EXITWC1uHmo', 'https://www.youtube.com/watch?v=rtXXUoaj0bU', 'https://www.youtube.com/watch?v=K5eD3ccxMUs', 'https://www.youtube.com/watch?v=UUtnh3J8xfQ', 'https://www.youtube.com/watch?v=XGc_GW57eQM', 'https://www.youtube.com/watch?v=i71J5vHvjzI', 'https://www.youtube.com/watch?v=Xb5wOgmMVSk', 'https://www.youtube.com/watch?v=wgPowY0G4Sw', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/watch?v=J1pa4irZwEM', 'https://www.youtube.com/watch?v=42fvJ5skggc', 'https://www.youtube.com/watch?v=gGPVIkSZ8lA', 'https://www.youtube.com/watch?v=2jwOJAAJ-ao', 'https://www.youtube.com/watch?v=-j32hc_pdi0', 'https://www.youtube.com/watch?v=SNpa5x4ihaU', 'https://www.youtube.com/watch?v=BrZ_3hsxWDs', 'https://www.youtube.com/watch?v=fj5uaLT-dh8', 'https://www.y

{'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/watch?v=q0I4f8771bc', 'https://www.youtube.com/shorts/CsaVgPJgKtg', 'https://www.youtube.com/watch?v=PRikXG8Mcrg', 'https://www.youtube.com/watch?v=EXITWC1uHmo', 'https://www.youtube.com/watch?v=rtXXUoaj0bU', 'https://www.youtube.com/watch?v=K5eD3ccxMUs', 'https://www.youtube.com/watch?v=UUtnh3J8xfQ', 'https://www.youtube.com/watch?v=XGc_GW57eQM', 'https://www.youtube.com/watch?v=i71J5vHvjzI', 'https://www.youtube.com/watch?v=Xb5wOgmMVSk', 'https://www.youtube.com/watch?v=wgPowY0G4Sw', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/watch?v=J1pa4irZwEM', 'https://www.youtube.com/watch?v=zLzRCWdXBpM', 'https://www.youtube.com/watch?v=42fvJ5skggc', 'https://www.youtube.com/watch?v=gGPVIkSZ8lA', 'https://www.youtube.com/watch?v=2jwOJAAJ-ao', 'https://www.youtube.com/watch?v=-j32hc_pdi0', 'https://www.youtube.com/watch?v=SNpa5x4ihaU', 'https://www.youtube.com/watch?v=BrZ_3hsxWDs', 'https://www.y

{'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/watch?v=q0I4f8771bc', 'https://www.youtube.com/shorts/CsaVgPJgKtg', 'https://www.youtube.com/watch?v=PRikXG8Mcrg', 'https://www.youtube.com/watch?v=EXITWC1uHmo', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/watch?v=rtXXUoaj0bU', 'https://www.youtube.com/watch?v=K5eD3ccxMUs', 'https://www.youtube.com/watch?v=UUtnh3J8xfQ', 'https://www.youtube.com/watch?v=XGc_GW57eQM', 'https://www.youtube.com/watch?v=i71J5vHvjzI', 'https://www.youtube.com/watch?v=Xb5wOgmMVSk', 'https://www.youtube.com/watch?v=_h3a6x-Z6VI', 'https://www.youtube.com/watch?v=wgPowY0G4Sw', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/watch?v=J1pa4irZwEM', 'https://www.youtube.com/watch?v=zLzRCWdXBpM', 'https://www.youtube.com/watch?v=42fvJ5skggc', 'https://www.youtube.com/watch?v=gGPVIkSZ8lA', 'https://www.youtube.com/watch?v=UHH01gYnzTY', 'https://www.youtube.com/watch?v=2jwOJAAJ-ao', 'https://www.y

{'https://www.youtube.com/watch?v=NXT6gUAhsis', 'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/watch?v=q0I4f8771bc', 'https://www.youtube.com/shorts/CsaVgPJgKtg', 'https://www.youtube.com/watch?v=PRikXG8Mcrg', 'https://www.youtube.com/shorts/sQUEyWJDIGI', 'https://www.youtube.com/watch?v=EXITWC1uHmo', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/watch?v=rtXXUoaj0bU', 'https://www.youtube.com/watch?v=K5eD3ccxMUs', 'https://www.youtube.com/watch?v=UUtnh3J8xfQ', 'https://www.youtube.com/watch?v=XGc_GW57eQM', 'https://www.youtube.com/watch?v=i71J5vHvjzI', 'https://www.youtube.com/watch?v=Xb5wOgmMVSk', 'https://www.youtube.com/watch?v=_h3a6x-Z6VI', 'https://www.youtube.com/watch?v=wgPowY0G4Sw', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/watch?v=J1pa4irZwEM', 'https://www.youtube.com/watch?v=zLzRCWdXBpM', 'https://www.youtube.com/watch?v=42fvJ5skggc', 'https://www.youtube.com/watch?v=7eR_vFnuA7k', 'https://www.yo

...

{'https://www.youtube.com/shorts/_17eDAifSc4', 'https://www.youtube.com/shorts/377GYuAfpJI', 'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/shorts/OQzgdr3XqaU', 'https://www.youtube.com/watch?v=mLKJyAN4Hww', 'https://www.youtube.com/watch?v=D7ecVbXCj98', 'https://www.youtube.com/watch?v=b8jG3HNqzfI', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/watch?v=PyKaH7XCdj0', 'https://www.youtube.com/watch?v=Djul65VUE-k', 'https://www.youtube.com/shorts/2ZdxaCc4tWM', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/_Q__xpB2AMk', 'https://www.youtube.com/watch?v=M-TEzsyxEfA', 'https://www.youtube.com/shorts/B-_MCPwo4IQ', 'https://www.youtube.com/shorts/8z2F-2FO7L0', 'https://www.youtube.com/shorts/nu_PDZnXFFU', 'https://www.youtube.com/shorts/KEcCUsVwNws', 'https://www.youtube.com/shorts/VK4jQhN_U7Y', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/shorts/gEsVSRO3zT8', 'https://www.youtube.com/

{'https://www.youtube.com/shorts/_17eDAifSc4', 'https://www.youtube.com/shorts/377GYuAfpJI', 'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/shorts/OQzgdr3XqaU', 'https://www.youtube.com/watch?v=mLKJyAN4Hww', 'https://www.youtube.com/watch?v=D7ecVbXCj98', 'https://www.youtube.com/watch?v=b8jG3HNqzfI', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/watch?v=PyKaH7XCdj0', 'https://www.youtube.com/watch?v=Djul65VUE-k', 'https://www.youtube.com/shorts/2ZdxaCc4tWM', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/_Q__xpB2AMk', 'https://www.youtube.com/watch?v=M-TEzsyxEfA', 'https://www.youtube.com/shorts/B-_MCPwo4IQ', 'https://www.youtube.com/shorts/8z2F-2FO7L0', 'https://www.youtube.com/shorts/nu_PDZnXFFU', 'https://www.youtube.com/shorts/KEcCUsVwNws', 'https://www.youtube.com/shorts/VK4jQhN_U7Y', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/shorts/gEsVSRO3zT8', 'https://www.youtube.com/

{'https://www.youtube.com/shorts/_17eDAifSc4', 'https://www.youtube.com/shorts/377GYuAfpJI', 'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/shorts/OQzgdr3XqaU', 'https://www.youtube.com/watch?v=mLKJyAN4Hww', 'https://www.youtube.com/watch?v=D7ecVbXCj98', 'https://www.youtube.com/watch?v=b8jG3HNqzfI', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/watch?v=PyKaH7XCdj0', 'https://www.youtube.com/watch?v=Djul65VUE-k', 'https://www.youtube.com/shorts/2ZdxaCc4tWM', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/_Q__xpB2AMk', 'https://www.youtube.com/watch?v=M-TEzsyxEfA', 'https://www.youtube.com/shorts/B-_MCPwo4IQ', 'https://www.youtube.com/shorts/8z2F-2FO7L0', 'https://www.youtube.com/shorts/nu_PDZnXFFU', 'https://www.youtube.com/shorts/KEcCUsVwNws', 'https://www.youtube.com/shorts/VK4jQhN_U7Y', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/shorts/gEsVSRO3zT8', 'https://www.youtube.com/

{'https://www.youtube.com/shorts/_17eDAifSc4', 'https://www.youtube.com/shorts/377GYuAfpJI', 'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/shorts/OQzgdr3XqaU', 'https://www.youtube.com/watch?v=mLKJyAN4Hww', 'https://www.youtube.com/watch?v=D7ecVbXCj98', 'https://www.youtube.com/watch?v=b8jG3HNqzfI', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/watch?v=PyKaH7XCdj0', 'https://www.youtube.com/watch?v=Djul65VUE-k', 'https://www.youtube.com/shorts/2ZdxaCc4tWM', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/_Q__xpB2AMk', 'https://www.youtube.com/watch?v=M-TEzsyxEfA', 'https://www.youtube.com/shorts/B-_MCPwo4IQ', 'https://www.youtube.com/shorts/8z2F-2FO7L0', 'https://www.youtube.com/shorts/nu_PDZnXFFU', 'https://www.youtube.com/shorts/KEcCUsVwNws', 'https://www.youtube.com/shorts/VK4jQhN_U7Y', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/shorts/gEsVSRO3zT8', 'https://www.youtube.com/

{'https://www.youtube.com/shorts/_17eDAifSc4', 'https://www.youtube.com/shorts/377GYuAfpJI', 'https://www.youtube.com/shorts/JKpTTTT9vQI', 'https://www.youtube.com/shorts/OQzgdr3XqaU', 'https://www.youtube.com/watch?v=mLKJyAN4Hww', 'https://www.youtube.com/watch?v=D7ecVbXCj98', 'https://www.youtube.com/watch?v=b8jG3HNqzfI', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/watch?v=PyKaH7XCdj0', 'https://www.youtube.com/watch?v=Djul65VUE-k', 'https://www.youtube.com/shorts/2ZdxaCc4tWM', 'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/_Q__xpB2AMk', 'https://www.youtube.com/watch?v=M-TEzsyxEfA', 'https://www.youtube.com/shorts/B-_MCPwo4IQ', 'https://www.youtube.com/shorts/8z2F-2FO7L0', 'https://www.youtube.com/shorts/nu_PDZnXFFU', 'https://www.youtube.com/shorts/KEcCUsVwNws', 'https://www.youtube.com/shorts/VK4jQhN_U7Y', 'https://www.youtube.com/watch?v=-e4fC_oXvMg', 'https://www.youtube.com/shorts/gEsVSRO3zT8', 'https://www.youtube.com/

In [ ]:
# MaC Core 개수 확인
#system_profiler | grep Cores

# CPU 개수
#system_profiler | grep Processors

#
#sysctl hw.ncpu

In [4]:
import pickle

f = open(CRAWLED_RES_SAVED_FILE, 'rb')
hrefURL = pickle.load(f)
f.close()

print(hrefURL)
print(len(hrefURL))

{'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/q-biGHZjjYo', 'https://www.youtube.com/watch?v=TA4amDsrqsk', 'https://www.youtube.com/shorts/68kv4wWRBho', 'https://www.youtube.com/watch?v=iP_vYvrqikI', 'https://www.youtube.com/watch?v=5z_-dlDOJBc', 'https://www.youtube.com/watch?v=ZZu9BjdYiLE', 'https://www.youtube.com/watch?v=nfbGqacJtmw', 'https://www.youtube.com/watch?v=EqnTzaYQGzA', 'https://www.youtube.com/watch?v=3VnvpIXmqqM', 'https://www.youtube.com/watch?v=SIRv-UeJZXM', 'https://www.youtube.com/watch?v=pnI1jkS1mNg', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/shorts/RbZwIBXKpj8', 'https://www.youtube.com/watch?v=gGPVIkSZ8lA', 'https://www.youtube.com/watch?v=78rKqGI4gdc', 'https://www.youtube.com/watch?v=VOd-3DaYF9I', 'https://www.youtube.com/watch?v=Nd0XanyY-2w', 'https://www.youtube.com/shorts/1ohRiJRBX4k', 'https://www.youtube.com/shorts/akE8v6GXsMs', 'https://www.youtube.com/watch?v=Djul65VUE-k', 'https://www.youtu

In [7]:
url = "https://www.youtube.com/results?search_query="

driver = webdriver.Chrome(r"C:\chromedriver.exe")
cnt = 0

search_url = url + "시고르자브종"
driver.get(url = search_url)

body = driver.find_element_by_tag_name('body')

hrefURL = set()

shelfContents = driver.find_elements_by_css_selector(
    "#contents > ytd-shelf-renderer > div > div > ytd-vertical-list-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
)

for item in shelfContents:
    if (item != ''):
        hrefURL.add(item.get_attribute('href'))

backup_height = driver.execute_script("return document.body.scrollHeight")

while cnt < 1000:
    body.send_keys(Keys.END)
    cnt += 1

allContents = driver.find_elements_by_css_selector(
    "#contents > ytd-item-section-renderer > div > ytd-video-renderer > div > ytd-thumbnail > a"
)

for item in allContents:
    if (item != ''):
        hrefURL.add(item.get_attribute('href'))

print(hrefURL)
print(len(hrefURL))

{'https://www.youtube.com/watch?v=2UxyjZtXysY', 'https://www.youtube.com/shorts/q-biGHZjjYo', 'https://www.youtube.com/watch?v=TA4amDsrqsk', 'https://www.youtube.com/shorts/68kv4wWRBho', 'https://www.youtube.com/watch?v=iP_vYvrqikI', 'https://www.youtube.com/watch?v=ZG-xBPCQDtw', 'https://www.youtube.com/watch?v=5z_-dlDOJBc', 'https://www.youtube.com/watch?v=nfbGqacJtmw', 'https://www.youtube.com/watch?v=f_Y3DQFlkoY', 'https://www.youtube.com/watch?v=SIRv-UeJZXM', 'https://www.youtube.com/watch?v=3VnvpIXmqqM', 'https://www.youtube.com/watch?v=EqnTzaYQGzA', 'https://www.youtube.com/shorts/QqHl_Aot2BY', 'https://www.youtube.com/shorts/RbZwIBXKpj8', 'https://www.youtube.com/watch?v=gGPVIkSZ8lA', 'https://www.youtube.com/watch?v=78rKqGI4gdc', 'https://www.youtube.com/watch?v=VOd-3DaYF9I', 'https://www.youtube.com/watch?v=Nd0XanyY-2w', 'https://www.youtube.com/shorts/1ohRiJRBX4k', 'https://www.youtube.com/shorts/akE8v6GXsMs', 'https://www.youtube.com/shorts/22mTeEk2t2M', 'https://www.youtub